# Project 3: Time-Series Forecasting & Volatility Analysis (AAPL)

## Data Download & Setup


In [ ]:
## Install packages 

import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

plt.style.use("seaborn-v0_8")


In [ ]:
## Download APPLE Stock 

stock = "AAPL"

data = yf.download(stock, start="2015-01-01", end="2025-01-01")

data.head()



In [ ]:
# Keep only Close column

prices = data["Close"].copy()
prices.head()


In [ ]:
## save to csv data

import os

os.makedirs("../data", exist_ok=True)
prices.to_csv("../data/aapl_prices.csv")


In [ ]:
## plot historic price 

prices.plot(figsize=(12, 6), title="AAPL Close Price (2015–2025)")
plt.ylabel("Price (USD)")
plt.show()


## Returns & Rolling Volatility

Goal:
Turn prices into daily returns, then measure how risky the series is over time using rolling volatility.

In [ ]:
prices = pd.read_csv("../data/aapl_prices.csv", index_col=0)
prices.index = pd.to_datetime(prices.index)

prices.head()


In [ ]:
# Daily returns: (P_t / P_{t-1}) - 1

returns = prices.pct_change()

# Drop the first NaN
returns = returns.dropna()

returns.head()


In [ ]:
## Plot daily Return 

returns.plot(figsize=(12, 4))
plt.title("AAPL Daily Returns")
plt.axhline(0, color="black", linewidth=0.8)
plt.ylabel("Daily Return")
plt.show()


In [ ]:
window = 21  # ~1 month

# Rolling daily volatility
rolling_vol_daily = returns.rolling(window=window).std()

# Annualized rolling volatility (more common in finance)
rolling_vol_annual = rolling_vol_daily * np.sqrt(252)

rolling_vol_annual.head(25)


In [ ]:
## Plot rolling annualized volatility

rolling_vol_annual.plot(figsize=(12, 4))
plt.title("AAPL Rolling Annualized Volatility (21-day window)")
plt.ylabel("Annualized Volatility")
plt.show()


In [ ]:
## Save returns & volatility to data/

returns.to_csv("../data/aapl_returns.csv")
rolling_vol_annual.to_csv("../data/aapl_rolling_vol_annual.csv")


## Build an ARIMA Forecast Model

Goal today
✔ Fit an ARIMA model on daily returns
✔ Make a short-term forecast
✔ Visualize it

## ARIMA Model

It predicts the next value in a time-series using:

Auto-Regressive part (past values)

Integrated part (differences)

MAving Average (past errors)

For returns, we often start with:

ARIMA(1,0,1)

Meaning:

1 lag of returns

no differencing (already stationary)

1 lag of error term

In [ ]:
from statsmodels.tsa.arima.model import ARIMA


In [ ]:
model = ARIMA(returns, order=(1, 0, 1))
results = model.fit()

print(results.summary())


In [ ]:
forecast_steps = 30

forecast = results.get_forecast(steps=forecast_steps)
mean_forecast = forecast.predicted_mean
conf_int = forecast.conf_int()



In [ ]:
plt.figure(figsize=(12,6))

plt.plot(returns[-200:], label="Historical Returns")
plt.plot(mean_forecast, label="Forecast", color="red")

plt.fill_between(
    conf_int.index,
    conf_int.iloc[:,0],
    conf_int.iloc[:,1],
    color="pink",
    alpha=0.3,
    label="Confidence Interval"
)

plt.title("ARIMA Forecast of AAPL Daily Returns")
plt.legend()
plt.show()

## Model Evaluation & Residual Analysis


🧪 Check if the ARIMA model is “good”

by analyzing:

✔ residuals (errors)
✔ autocorrelation
✔ distribution shape
✔ whether markets show fat tails & volatility clustering

In [ ]:
## Get the residuals

## Residuals = actual returns − model forecast
## If residuals are random → model is good
## If not → model is missing structure

residuals = results.resid

residuals.name = "Residuals"

residuals.head()


In [ ]:
## Plot residuals over time
    
residuals.plot(figsize=(12,4))
plt.title("ARIMA Residuals Over Time")
plt.axhline(0, color="black", linewidth=0.8)
plt.show()


In [ ]:
## Histogram of residuals

residuals.hist(bins=50, figsize=(8,4))
plt.title("Residuals Distribution")
plt.show()


In [ ]:
## Check ACF (autocorrelation)

from statsmodels.graphics.tsaplots import plot_acf

plot_acf(residuals, lags=30)
plt.title("Residuals Autocorrelation")
plt.show()


In [ ]:
residuals.describe()


## Forecast-Based Value-at-Risk (VaR)

Goal: 

✔ use your ARIMA forecast results
✔ estimate distribution of future returns
✔ calculate 95% Value-at-Risk (VaR)
✔ compare to historical VaR
✔ write conclusions

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

residuals = results.resid
forecast = results.get_forecast(steps=30)
mean_forecast = forecast.predicted_mean


We assume:

future returns ≈ normal

with

mean = model forecast

std = std of residuals

In [ ]:
## residual volatility 

sigma = residuals.std()
sigma


In [ ]:
## Compute 95% daily VaR

var_95 = - (mean_forecast + 1.65 * sigma)
var_95


In [ ]:
## Plot VaR vs Expected Return 

plt.figure(figsize=(12,6))

plt.plot(mean_forecast, label="Expected Return")
plt.plot(var_95, label="95% VaR", color="red")

plt.title("Forecast Expected Return vs 95% VaR")
plt.legend()
plt.show()


In [ ]:
## Historical VaR (for comparison)

hist_var_95 = residuals.quantile(0.05)
hist_var_95


In [ ]:
## Print a summary

print("Residual std: {:.2%}".format(sigma))
print("Historical 95% VaR: {:.2%}".format(hist_var_95))
print("Average Forecast 95% VaR: {:.2%}".format(var_95.mean()))


## INTERPRETATION


- ARIMA(1,0,1) on AAPL daily returns produced an expected return close to 0, consistent with weak-form market efficiency.
- Residual analysis showed heavy tails and volatility clustering, indicating that risk is time-varying and extreme moves are more common than under a normal distribution.
- Using residual volatility, the model estimated a 95% daily VaR of approximately –3%, meaning that on about 1 out of 20 days, losses are expected to exceed ~3%.
- Forecast VaR (-3.06%) was slightly more conservative than historical VaR (–2.89%), suggesting modestly elevated risk in the forecast period.

This project demonstrates how time-series models (ARIMA) and risk concepts (volatility, VaR) can be combined to produce forward-looking risk estimates for equity markets.



In [ ]:
## Save Results

pd.DataFrame({
    "expected_return": mean_forecast,
    "var_95": var_95
}).to_csv("../data/arima_var_forecast.csv")
